### Model Architecture: U-Net for Microaneurysm Detection<br>
Architecture Overview: The U-Net architecture was chosen for this project due to its proven effectiveness in medical image segmentation tasks, particularly in scenarios where precise localization of features like microaneurysms is crucial. U-Net is a fully convolutional network (FCN) that excels in handling small datasets and provides high accuracy in segmenting complex structures within images.<br>

#### Flow of Data within the Model:
 The U-Net architecture consists of two main parts: the contracting path (encoder) and the expanding path (decoder).
<br>

#### Contracting Path (Encoder):<br>

The encoder is responsible for capturing the context of the input image. It follows a typical convolutional neural network (CNN) structure, comprising repeated application of two 3x3 convolutional layers followed by a ReLU activation and a 2x2 max-pooling operation for downsampling.<br>
Each convolution operation doubles the number of feature channels, allowing the network to capture increasingly abstract and high-level features as the spatial dimensions of the image reduce.<br>
#### Bottleneck:
<br>
At the bottleneck of the network, the image is represented by a highly abstract feature map with a reduced spatial dimension but a rich feature set. This stage consists of two 3x3 convolutions with ReLU activations, followed by another 2x2 max-pooling operation<br> 

#### Expanding Path (Decoder):

The decoder is designed to recover the spatial resolution of the image, effectively upsampling the encoded features back to the original input size. It uses transposed convolutions (also known as deconvolutions) to upsample the feature maps.<br>
Each upsampling step is followed by a concatenation with the corresponding feature map from the encoder (skip connections). These skip connections allow the network to retain high-resolution information that might otherwise be lost during downsampling.<br>
#### Output Layer:<br>

The final layer is a 1x1 convolution that reduces the number of output channels to the desired number of classes (in this case, one for microaneurysms), followed by a sigmoid activation function to produce a pixel-wise binary classification map.
Choice of Optimizers and Metrics:
<br>

#### Optimizer:
<br>
The Adam optimizer was chosen for this model due to its adaptive learning rate capabilities, which help in faster convergence and robustness to varying gradient scales. Adam combines the advantages of two other popular optimizers—AdaGrad and RMSProp—making it particularly effective for training deep networks like U-Net.
<br>

#### Loss Function:
<br>
Binary Cross-Entropy (BCE): Given the binary nature of the task (microaneurysm present or not), Binary Cross-Entropy is used as the loss function. BCE is well-suited for binary classification tasks, providing a measure of dissimilarity between the predicted and true binary labels.









In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import glob

# Paths
dataset_path = "F:/Fyp/Preprocessing/1_Microaneurysms/Processed"
image_dir = os.path.join(dataset_path, "Images")
mask_dir = os.path.join(dataset_path, "Mask")

# Image dimensions
IMG_HEIGHT = 256
IMG_WIDTH = 256
IMG_CHANNELS = 3

# Load images and masks
def load_images_and_masks(image_dir, mask_dir):
    images = sorted(glob.glob(os.path.join(image_dir, "*")))
    masks = sorted(glob.glob(os.path.join(mask_dir, "*")))

    images = [cv2.imread(img) for img in images if os.path.isfile(img)]
    masks = [cv2.imread(mask, cv2.IMREAD_GRAYSCALE) for mask in masks if os.path.isfile(mask)]

    # Resize images and masks to the desired size
    images = [cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT)) for img in images]
    masks = [cv2.resize(mask, (IMG_WIDTH, IMG_HEIGHT)) for mask in masks]

    images = np.array(images)
    masks = np.expand_dims(np.array(masks), axis=-1)  # Add channel dimension

    # Normalize images and masks
    images = images / 255.0
    masks = masks / 255.0

    return images, masks

# Define U-Net model
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = Input(input_size)

    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Load the data
images, masks = load_images_and_masks(image_dir, mask_dir)

# Split data into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(images, masks, test_size=0.2, random_state=42)

# Data augmentation
data_gen_args = dict(rotation_range=15,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.1,
                     horizontal_flip=True,
                     fill_mode='nearest')

image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)

# Create generators
train_image_generator = image_datagen.flow(train_images, batch_size=16, seed=42)
train_mask_generator = mask_datagen.flow(train_masks, batch_size=16, seed=42)

# Create the combined dataset from the generators
def generator_to_dataset(image_gen, mask_gen):
    dataset = tf.data.Dataset.from_generator(
        lambda: zip(image_gen, mask_gen),
        output_signature=(
            tf.TensorSpec(shape=(16, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=tf.float32),
            tf.TensorSpec(shape=(16, IMG_HEIGHT, IMG_WIDTH, 1), dtype=tf.float32)
        )
    )
    return dataset

train_dataset = generator_to_dataset(train_image_generator, train_mask_generator)

# Apply prefetching to the dataset
train_dataset = train_dataset.repeat().prefetch(buffer_size=tf.data.AUTOTUNE)

# Create the U-Net model
model = unet_model()

# Train the model
history = model.fit(train_dataset,
                    steps_per_epoch=len(train_images) // 16,
                    validation_data=(val_images, val_masks),
                    epochs=1)

# Save the trained model
model.save('unet_model.h5')

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_images, val_masks)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 3180s 157s/step - accuracy: 0.9634 - loss: 0.8835 - val_accuracy: 0.9892 - val_loss: 0.0281


3/3 ━━━━━━━━━━━━━━━━━━━━ 125s 38s/step - accuracy: 0.9902 - loss: 0.0250
Validation Accuracy: 98.92%
